<a href="https://colab.research.google.com/github/BroccoliWarrior/transformer-basic-knowledge/blob/main/Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Normalization***

purpose：标准化隐藏层的输出，稳定训练过程，缓解梯度消失或爆炸的问题，同时加速收敛并提高模型的泛化能力



### ***几种常见的Normalization***

    * Batch Normalization
    * Layer Normalization
    * Instance Normalization
    * Group Normalization

1. **Batch Normalization**

  在激活函数之前，对每个channel在batch维度上计算均值与方差，并将激活值约束到均值为0、方差为1的分布，从而***减少ICS问题，并加速收敛***

    * ICS（Internal Covariate Shift）：随着训练的进行，网络参数不断更新，这会导致每一层的输入数据的分布发生改变。例如，在神经网络的第一层输入数据可能服从某种分布，但是经过第一层的变换后，第二层的输入数据分布就会发生变化，而且这种变化会随着网络层数的增加不断累积
    * 这种输入分布的变化会使得模型训练变得困难

  **计算过程**

  假设某一层的输出维度为[N,C,H,W]，其中N表示batch_size，C表示通道数，H和W分别表示特征图的高和宽

  step1：计算均值

  $\mu_c = \frac{1}{N \times H \times W} \sum_{n=1}^{N} \sum_{h=1}^{H} \sum_{w=1}^{W} x_{n,c,h,w}$

  step2：计算方差

  $\sigma_c^2 = \frac{1}{N \times H \times W} \sum_{n=1}^{N} \sum_{h=1}^{H} \sum_{w=1}^{W} (x_{n,c,h,w} - \mu_c)^2$

  step3：归一化

  $\hat{x}_{n,c,h,w} = \frac{x_{n,c,h,w} - \mu_c}{\sqrt{\sigma_c^2 + \epsilon}}$

  step4：缩放和平移

  $y_{n,c,h,w} = \gamma_c \hat{x}_{n,c,h,w} + \beta_c$

  其中 $\gamma_c$ 和 $\beta_c$ 为可学习参数（初始时常设为 $\gamma_c = 1$，$\beta_c = 0$）

  **作用**

    * 减少ICS，避免梯度消失或爆炸
    * 允许使用较大的学习率
    * 降低对权重初始化的敏感度
    * 对隐藏层输出有轻微的正则化效果（在训练过程中，BN 是基于一个 mini-batch 的数据来计算均值和方差进行归一化操作的。由于不同 mini-batch 的数据存在差异，这就导致每次归一化的结果会有一定的波动。对于网络中的某一层来说，这种波动类似于给输入数据加入了噪声。模型为了适应这种噪声，就需要学习更加鲁棒的特征表示，从而提高了模型的泛化能力 ，起到了类似正则化的效果。）

  2. **Layer Normalization**

  不依赖batch维度，在单个样本内对特征进行归一化

  **计算过程**

  给定输入向量$x=(x_1,x_2,⋯,x_n)$，LN统计所有元素的均值与方差

  $y = \frac{x - \text{E}(x)}{\sqrt{\text{Var}(x) + \epsilon}} * \gamma + \beta$

  其中$\text{E}(x) = \frac{1}{n} \sum_{i=1}^{n} x_i$，$\text{Var}(x) = \frac{1}{n} \sum_{i=1}^{n} (x_i - \text{E}(x))^2$

    * 这里是“有偏估计”

In [ ]:
import torch
from torch import nn

class LayerNorm(nn.Module):
  def __init__(self, features, eps=1e-6) -> None:
    super(LayerNorm, self).__init__()

    self.gamma = nn.Parameter(torch.ones(features))
    self.beta = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True, unbiased=False)
    return self.gamma * (x - mean) / (std + self.eps) + self.beta

  3. **Instance Normalization**

  IN最初用于图像风格迁移任务，在每张图像的特征图上分别做归一化

  **计算过程**

  对输入$x∈\mathbb {R} ^{N×C×H×W}$，IN在每个样本n、每个通道c的特征图上计算均值和方差

  step1：计算均值

  $\mu_{n,c} = \frac{1}{HW} \sum_{h=1}^{H} \sum_{w=1}^{W} x_{n,c,h,w}$

  step2：计算方差

  $\quad \sigma_{n,c}^2 = \frac{1}{HW}\sum_{h=1}^{H} \sum_{w=1}^{W} (x_{n,c,h,w} - \mu_{n,c})^2$

  step3：归一化

  $y_{n,c,h,w} = \frac{x_{n,c,h,w} - \mu_{n,c}}{\sqrt{\sigma_{n,c}^2 + \epsilon}} \quad $

  4. **Group Normalization**

  主要为了**解决BN对batch size依赖较大**

  **计算过程**

  将$C$个通道，分为$G$组，则每组有$C/G$个通道，在单个样本的特征图中，对同一组的所有通道的其对于的空间位置$H×W$做均值和方差计算

  $\mu_{n,g} = \frac{1}{(C/G)HW} \sum_{c=g\frac{C}{G}}^{(g+1)\frac{C}{G} - 1} \sum_{h=1}^{H} \sum_{w=1}^{W} x_{n,c,h,w}$

  $\sigma_{n,g} = \sqrt{\frac{1}{(C/G)HW} \sum_{c=g\frac{C}{G}}^{(g+1)\frac{C}{G} - 1} \sum_{h=1}^{H} \sum_{w=1}^{W} (x_{n,c,h,w} - \mu_{n,g})^2 + \epsilon}$

***Layer Normalization的优化方案***

1. Root Mean Square（RMS） Layer Normalization

  与LN对比起来，RMS Norm不需要进行$x-μ$，只用：

  $\text{RMS}(\mathbf{a}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} a_i^2}$

  $\bar{a}_i = \frac{a_i}{\text{RMS}(\mathbf{a})} \cdot g_i$

  其中$g_i$是可学习的缩放系数

2. pRMS Norm

  RMS Norm对整个向量都要计算$RMS$，当面对大向量或特指数极多的情况，计算量增大，于是提出pRMS Norm，仅使用前$p%$的元素计算$RMS$

## ***Layer Normalization和Instance Normalization***

  1. **区别**

  * LayerNorm 在**单个样本内部的特征维**上做标准化
  * InstanceNorm 在**单个样本的每个通道的空间位置**上做标准化

  **Layer Norm**：假设 N=1, L=3, D=4，输入形状为 (1, 3, 4)：

    token1: [1.0, 2.0, 3.0, 4.0]

    token2: [2.0, 4.0, 6.0, 8.0]
  
    token3: [1.5, 3.0, 4.5, 6.0]

  对每个 token 的 4 维向量单独作为一个集合做归一化（每行单独处理），确保同一 token 内部的特征尺度一致

  **Instance Norm**：假设 N=1, C=1, H=2, W=2，灰度图像，像素矩阵：

    [[1.0, 2.0],
    
    [3.0, 4.0]]

  把这张图该通道的 4 个像素作为一个集合做归一化，使得该通道内的像素分布在该图上被标准化
